<a href="https://colab.research.google.com/github/iankim20/MAIC-AI-Challenge_hypotension-prediction/blob/main/Aggregate_and_tfrec_for_LSTM_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/MAIC_data"

/content/drive/My Drive/MAIC_data


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random as random

import pandas as pd
import numpy as np
from pylab import rcParams

import tensorflow as tf
from keras import optimizers, Sequential
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [ ]:
import gc
import os, sys, math

# Three types of data can be stored in TFRecords: bytestrings, integers and floats
# They are always stored as lists, a single data element will be a list of size 1
def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
    return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))
  

def to_tfrecord(tfrec_filewriter, bp, event):  
#     class_num = np.argmax(np.array(CLASSES)==label) # 'roses' => 2 (order defined in CLASSES)
#     one_hot_class = np.eye(len(CLASSES))[class_num]     # [0, 0, 1, 0, 0] for class #2, roses

    feature = {
        "bp": _float_feature(bp.numpy().tolist()), # one image in the list
#       "class": _int_feature([class_num]),        # one class in the list

      # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
        "event":         _float_feature(event.numpy().tolist()),          # fixed length (1) list of strings, the text label
#       "size":          _int_feature([height, width]),         # fixed length (2) list of ints
#       "one_hot_class": _float_feature(one_hot_class.tolist()) # variable length  list of floats, n=len(CLASSES)
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
index=np.array([500,1000,1500,2000,2500,3000])  ## index=0 인 x_train은 validation의 nonevent로 사용할 것임!! (20만개 nonevent for validation set)
# x_train_full=np.load('processed_data/x_train_nonevent_3000.npz')['arr_0']
for val in index:
    tmp_x = np.load('processed_data/x_train_nonevent_{}.npz'.format(val))['arr_0']
    tmp_y = np.load('processed_data/y_train_nonevent_{}.npz'.format(val))['arr_0']
    tmp_y = tmp_y*1

    idx = np.random.permutation(len(tmp_x))

    x_train_patch = tmp_x[idx][:,5:]
    y_train_patch = tmp_y[idx]
    y_train_patch = y_train_patch[...,None]    

    x_train_patch -= 20
    x_train_patch /= 180

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_patch, y_train_patch))
    
    # print("Tensorflow version " + tf.__version__)
    AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

    SHARDS = 40
    shard_size = math.ceil(1.0 * len(train_dataset) / SHARDS)

    train_dataset2 = train_dataset.batch(shard_size) # sharding: there will be one "batch" of images per file 
      
    print("Writing TFRecords")
    for shard, (bp, event) in enumerate(train_dataset2):
        # batch size used as shard size here
        shard_size = bp.shape[0]
        # good practice to have the number of records in the filename
        filename = "processed_data/tfrec/train_nonevent_"+"{:02d}_{:02d}-{}.tfrec".format(int(val/500),shard, shard_size)

        with tf.io.TFRecordWriter(filename) as out_file:
            for i in range(shard_size):
                example = to_tfrecord(out_file,
                                      bp[i], # re-compressed image: already a byte string
                                      event[i])
                out_file.write(example.SerializeToString())
            print("Wrote file {} containing {} records".format(filename, shard_size))

    # x_train_full=np.concatenate((x_train_full,tmp_x),axis=0)
    print("now x at %s"% val)
    gc.collect()

Writing TFRecords
Wrote file processed_data/tfrec/train_nonevent_01_00-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_01-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_02-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_03-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_04-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_05-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_06-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_07-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_08-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_09-5403.tfrec containing 5403 records
Wrote file processed_data/tfrec/train_nonevent_01_10-5403.tfrec containing 5403 records
Wrote file pro

In [ ]:
x_train_full.shape

(112122, 2005)

In [ ]:
# y_train_full=[]
# for val in index:
#     tmp_y = np.load('/kaggle/input/0927-nonan-processed/y_train_{}.npz'.format(val))['arr_0']
#     y_train_full=np.concatenate((y_train_full,tmp_y))
#     print("now y at %s"% val)
    
#     gc.collect()

In [ ]:
x_val_nonevent = np.load('processed_data/x_train_nonevent_0.npz')['arr_0']
y_val_nonevent = np.load('processed_data/y_train_nonevent_0.npz')['arr_0']
y_val_nonevent = y_val_nonevent *1

x_val_nonevent = x_val_nonevent[:,5:]

x_val_nonevent -= 20
x_val_nonevent /= 180


In [ ]:
y_val_nonevent = y_val_nonevent[...,None]
val_nonevent_dataset = tf.data.Dataset.from_tensor_slices((x_val_nonevent, y_val_nonevent))

# print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

SHARDS = 40
shard_size = math.ceil(1.0 * len(val_nonevent_dataset) / SHARDS)

val_nonevent_dataset2 = val_nonevent_dataset.batch(shard_size) # sharding: there will be one "batch" of images per file 
  
print("Writing TFRecords")
for shard, (bp, event) in enumerate(val_nonevent_dataset2):
    # batch size used as shard size here
    shard_size = bp.shape[0]
    # good practice to have the number of records in the filename
    filename = "processed_data/tfrec/val_nonevent_"+"{:02d}-{}.tfrec".format(shard, shard_size)

    with tf.io.TFRecordWriter(filename) as out_file:
        for i in range(shard_size):
            example = to_tfrecord(out_file,
                                  bp[i], # re-compressed image: already a byte string
                                  event[i])
            out_file.write(example.SerializeToString())
        print("Wrote file {} containing {} records".format(filename, shard_size))

Writing TFRecords
Wrote file processed_data/tfrec/val_nonevent_00-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_01-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_02-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_03-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_04-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_05-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_06-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_07-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_08-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_09-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_10-5227.tfrec containing 5227 records
Wrote file processed_data/tfrec/val_nonevent_11-5227.tfrec containing

In [ ]:
# x_train_400000 = np.load('processed_data/x_train_full.npz')['arr_0']
# y_train_400000 = np.load('processed_data/y_train_full.npz')['arr_0']

In [ ]:
# x_val_event = x_train_400000[y_train_400000==1]
# x_val_event = x_val_event[:,5:]

# idx = np.random.permutation(len(x_val_event))
# x_val_event_20000 = x_val_event[idx][:20000]

# x_val_event_20000 -= 20
# x_val_event_20000 /= 180

# np.savez_compressed('processed_data/x_val_event_20000.npz', x_val_event_20000)

# y_val_event = np.full((20000,),1)
# y_val_nonevent = y_val_nonevent *1

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
x_val_event_20000 = np.load('processed_data/x_val_event_20000.npz')['arr_0']

y_val_event = np.full((20000,),1)
y_val_nonevent = y_val_nonevent *1

In [ ]:
x_val = np.vstack((x_val_nonevent,x_val_event_20000))
y_val = np.hstack((y_val_nonevent,y_val_event))
y_val = y_val[...,None]

print(x_val.shape)
print(y_val.shape)

(229046, 2000)
(229046, 1)


In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))

# print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

SHARDS = 40
shard_size = math.ceil(1.0 * len(val_dataset) / SHARDS)

val_dataset2 = val_dataset.batch(shard_size) # sharding: there will be one "batch" of images per file 
  
print("Writing TFRecords")
for shard, (bp, event) in enumerate(val_dataset2):
    # batch size used as shard size here
    shard_size = bp.shape[0]
    # good practice to have the number of records in the filename
    filename = "processed_data/tfrec/val_"+"{:02d}-{}.tfrec".format(shard, shard_size)

    with tf.io.TFRecordWriter(filename) as out_file:
        for i in range(shard_size):
            example = to_tfrecord(out_file,
                                  bp[i], # re-compressed image: already a byte string
                                  event[i])
            out_file.write(example.SerializeToString())
        print("Wrote file {} containing {} records".format(filename, shard_size))

Writing TFRecords
Wrote file processed_data/tfrec/val_00-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_01-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_02-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_03-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_04-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_05-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_06-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_07-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_08-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_09-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_10-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_11-5727.tfrec containing 5727 records
Wrote file processed_data/tfrec/val_12-5727.tfrec containing 5727 records
Wrote file processed